Be it in the realm of Bioinformatics or in the domain of Databases, the command line tool `awk` proves to be a great utility when processing and analysing large files. 

In [1]:
pwd

/Users/debangana/Desktop/Bash_programming


In [2]:
cd /Users/debangana/linux_basics/tutorials;ls #execute one bash command after another with a semicolon
 

DMP.csv				calculator.sh
GSE111629Annotation1.csv	student_data.txt


Let us try to perform some processing on the file `GSE111629Annotation1.csv`. Then we shall discuss some applications of the `awk` tool using the `GSE111629Annotation1.csv` file

Display the number of rows in the file

In [70]:
cat GSE111629Annotation1.csv | wc -l #displays number of records in the file icluding header

     573


In [71]:
tail -n +2 GSE111629Annotation1.csv | wc -l # another way to display the nuber of records in the file excluding the header

     572


Display the number of columns/fields using awk

In [73]:
head -1 GSE111629Annotation1.csv | awk -F',' '{print NF}'
# The command selects the header, pipes it to awk. NF refers to Number of fields in the header and then we print the NF

11


Display the fields and mark each of them with an index to aid in easy extraction/filtration

In [11]:
head -n 1 GSE111629Annotation1.csv | tr ',' '\n' | nl 
# head -n 1: extracts the first line '-n' specifies the number of lines to be displayes.
# tr ',' '\n': replaces commas with newlines, listing one column per line. 
# nl adds line numbers (column indexes)




     1	"title"
     2	"geo_accession"
     3	"submission_date"
     4	"last_update_date"
     5	"age:ch1"
     6	"disease state:ch1"
     7	"ethnicity:ch1"
     8	"gender:ch1"
     9	"tissue:ch1"
    10	"Sentrix_ID"
    11	"Sentrix_Position"


In [10]:
# Displaying the column names with the indexes makes it easier to extract the columns with cut command
# the '>' sign redirects the filtered columns to a new file 
cut -d',' -f1,10 GSE111629Annotation1.csv > GSE111629Annotation1Filtered.csv 

head -n 6 GSE111629Annotation1Filtered.csv # displays top 6 lines from the newly created file with the filtered columns

"title","Sentrix_ID"
"3999979001_R01C01","3999979001"
"3999979001_R01C02","3999979001"
"3999979001_R02C01","3999979001"
"3999979001_R02C02","3999979001"
"3999979001_R03C01","3999979001"


Now let us perform the filtering task using the `awk` tool

In [15]:
#Check the column indices from the output of the head command
awk -F ',' '{print $1,$3,$10}' GSE111629Annotation1.csv > GSE111629Annotation1Filtered1.csv # filters the column with the specified indices from the file
# -F specifies the field separator
head -n 6 GSE111629Annotation1Filtered1.csv

"title" "submission_date" "Sentrix_ID"
"3999979001_R01C01" "Mar 09 2018" "3999979001"
"3999979001_R01C02" "Mar 09 2018" "3999979001"
"3999979001_R02C01" "Mar 09 2018" "3999979001"
"3999979001_R02C02" "Mar 09 2018" "3999979001"
"3999979001_R03C01" "Mar 09 2018" "3999979001"


Let us discuss some concepts of the `awk` tool in detail. We often come across `$NF` with `awk`. Let us understand `$NF` in detail.
Let us consider the file `GSE111629Annotation1.csv` to understand this in detail

In [31]:
awk -F ',' '{ print $NF }' GSE111629Annotation1.csv | head -n 3  # $NF prints the value of the last field
#In the above command we have used the pipe to display only the first three values of the otherwise long output

"Sentrix_Position"
"R01C01"
"R01C02"


In [32]:
awk -F ',' 'NR > 1 { print $NF }' GSE111629Annotation1.csv | head -n 3 # NR > 1 specifies that we want to exclude the header "Sentrix_Position"

"R01C01"
"R01C02"
"R02C01"


Now let us print the last field exclduing the header along with line numbers

In [41]:
awk -F ',' ' NR > 1 { print NR ":" $NF }' GSE111629Annotation1.csv | head -n 3


2:"R01C01"
3:"R01C02"
4:"R02C01"


In [43]:
awk -F ',' '{ print NR ":" $NF }' GSE111629Annotation1.csv | head -n 3 # header is printed when we don't specify NR > 1


1:"Sentrix_Position"
2:"R01C01"
3:"R01C02"


Now let us proceed to some advanced data processing using awk. Suppose we want to print the `Sentrix_ID` and `age` where `age > 50` from the file `GSE111629Annotation1.csv`

In [61]:
head -n 4 GSE111629Annotation1.csv # We see that the 5th column is age:ch1

#Now let us print the number of records for which age > 50
awk -F',' 'NR > 1 && ($5 > 50) {print $10, $5 }' GSE111629Annotation1.csv | wc -l 


"title","geo_accession","submission_date","last_update_date","age:ch1","disease state:ch1","ethnicity:ch1","gender:ch1","tissue:ch1","Sentrix_ID","Sentrix_Position"
"3999979001_R01C01","GSM3035401","Mar 09 2018","Mar 10 2018","74","Parkinson's disease (PD)","Caucasian","Female","whole blood","3999979001","R01C01"
"3999979001_R01C02","GSM3035402","Mar 09 2018","Mar 10 2018","73","PD-free control","Caucasian","Female","whole blood","3999979001","R01C02"
"3999979001_R02C01","GSM3035403","Mar 09 2018","Mar 10 2018","62","Parkinson's disease (PD)","Caucasian","Male","whole blood","3999979001","R02C01"
       0


We see that the number of records where age > 50 is 0, however, this is not the case. To see the age of every record, let us use the `cut` command that we already used above

In [62]:
cut -d',' -f5 GSE111629Annotation1.csv | head -n 10

"age:ch1"
"74"
"73"
"62"
"72"
"72"
"69"
"55"
"79"
"81"


The problem of getting 0 records using `awk` arises because of the presence of quotes around the numbers, hence awk sees these entries as strings and is not able to perform the numeric comparison. Here is a solution for the above problem using `awk` and `gsub`

In [68]:
awk -F',' 'NR > 1 { gsub(/"/, "", $5); if ($5 > 50) print $10, $5 }' GSE111629Annotation1.csv | wc -l #displays number of records which have age > 50

     518


In [69]:
awk -F',' 'NR > 1 { gsub(/"/, "", $5); if ($5 < 50) print $10, $5 }' GSE111629Annotation1.csv | wc -l #displays number of records where age < 50

      49


The gsub function used in the above command is a string manipulation tool that performs global substitution. It replaces all occurrences of a specified pattern within a field. In our example, `gsub(/"/, "", $5)` removes all double quotes from the 5th column ($5, which represents age). This is important because quoted numbers are treated as strings, and removing the quotes ensures proper numerical comparison.

We will explore gsub and other powerful regular expression features in detail in upcoming tutorials.